# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vrangel',42.7297,133.0839,19.03,98,100,1.07,RU,1692024460
1,1,Adamstown,-25.0660,-130.1015,20.72,76,78,9.94,PN,1692024460
2,2,Waitangi,-43.9535,-176.5597,12.12,96,49,1.34,NZ,1692024461
3,3,Alofi,-19.0595,-169.9187,22.94,94,75,3.09,NU,1692024462
4,4,Jamestown,42.0970,-79.2353,21.96,77,0,1.54,US,1692024262


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo=True, alpha=0.2,
    size="Humidity", color="City", tiles='OSM')

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#get the ideal temp
good_temp_df = city_data_df.loc[(city_data_df["Max Temp"]<27.00)&(city_data_df["Max Temp"]>21.00)]
#get the ideal wind speed
good_wind_df = good_temp_df.loc[good_temp_df["Wind Speed"]<4.5]
#get the ideal cloudiness
good_clouds_df = good_wind_df.loc[good_wind_df["Cloudiness"]==0]

# Drop any rows with null values
ideal_df = good_clouds_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Jamestown,42.0970,-79.2353,21.96,77,0,1.54,US,1692024262
246,246,Visby,57.6409,18.2960,25.02,53,0,4.12,SE,1692024403
250,250,Upington,-28.4478,21.2561,24.21,16,0,2.57,ZA,1692024594
354,354,Mujiayingzi,42.1167,118.7833,22.75,54,0,2.73,CN,1692024660
369,369,Hancheng,32.5186,112.3522,25.13,72,0,2.01,CN,1692024670
412,412,Yarmouth,41.7057,-70.2286,24.80,74,0,3.09,US,1692024696
472,472,Chifeng,42.2683,118.9636,23.41,50,0,2.25,CN,1692024727
491,491,Chimoio,-19.1164,33.4833,25.50,26,0,2.40,MZ,1692024738
493,493,Vernal,40.4555,-109.5287,22.03,33,0,2.06,US,1692024739
548,548,Nazca,-14.8333,-74.9500,25.35,41,0,2.47,PE,1692024773


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,Jamestown,US,42.0970,-79.2353,77,
246,Visby,SE,57.6409,18.2960,53,
250,Upington,ZA,-28.4478,21.2561,16,
354,Mujiayingzi,CN,42.1167,118.7833,54,
369,Hancheng,CN,32.5186,112.3522,72,
412,Yarmouth,US,41.7057,-70.2286,74,
472,Chifeng,CN,42.2683,118.9636,50,
491,Chimoio,MZ,-19.1164,33.4833,26,
493,Vernal,US,40.4555,-109.5287,33,
548,Nazca,PE,-14.8333,-74.9500,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Jamestown - nearest hotel: DoubleTree Jamestown
Visby - nearest hotel: Hotell Gute
Upington - nearest hotel: Die Eiland Holiday Ressort
Mujiayingzi - nearest hotel: No hotel found
Hancheng - nearest hotel: No hotel found
Yarmouth - nearest hotel: The Inn At Cape Cod
Chifeng - nearest hotel: 心连心酒店
Chimoio - nearest hotel: Hotel Amirana
Vernal - nearest hotel: Landmark Inn and Suites
Nazca - nearest hotel: CASA HACIENDA


,City,Country,Lat,Lng,Humidity,Hotel Name
4,Jamestown,US,42.0970,-79.2353,77,DoubleTree Jamestown
246,Visby,SE,57.6409,18.2960,53,Hotell Gute
250,Upington,ZA,-28.4478,21.2561,16,Die Eiland Holiday Ressort
354,Mujiayingzi,CN,42.1167,118.7833,54,No hotel found
369,Hancheng,CN,32.5186,112.3522,72,No hotel found
412,Yarmouth,US,41.7057,-70.2286,74,The Inn At Cape Cod
472,Chifeng,CN,42.2683,118.9636,50,心连心酒店
491,Chimoio,MZ,-19.1164,33.4833,26,Hotel Amirana
493,Vernal,US,40.4555,-109.5287,33,Landmark Inn and Suites
548,Nazca,PE,-14.8333,-74.9500,41,CASA HACIENDA


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
humidity_map = hotel_df.hvplot.points("Lng", "Lat", geo=True, alpha=0.2,
    size="Humidity", color="City", tiles='OSM',hover_cols=["Country","Hotel Name"])

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)